In [1]:
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import os
import json


# jupyter_bbox_widget

It's a custom widget that helps you annotate images for object detection tasks.

## Introduction

Initialize the widget with an image path and a list of classes.

Click and drag anywhere on the image to create bboxes, move and resize them as necessary.


In [2]:
widget = BBoxWidget(
    image='fruit/fruit.jpg',
    classes=['apple', 'orange', 'pear'],
)
widget

BBoxWidget(classes=['apple', 'orange', 'pear'], colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',…

Access the current state of bboxes like this:

In [3]:
widget.bboxes

[]

In [5]:
from pathlib import Path
import base64

data = Path('fruit/fruit2.jpg').read_bytes()
encoded = str(base64.b64encode(data), 'utf-8')
widget.image = "data:image/jpg;base64,"+encoded

## A simple annotation workflow

Let's say we have a folder of image files that we would like to create annotations for.

In [ ]:
path = 'fruit'
files = sorted(os.listdir(path))

annotations = {}
annotations_path = 'annotations.json'

We'll use a `BBoxWidget` for creating annotations for an image. It already has "Submit" and "Skip" buttons for going through our list of images. Let's also add a progress bar - from the `ipywidgets` library.

In [ ]:
# a progress bar to show how far we got
w_progress = widgets.IntProgress(value=0, max=len(files), description='Progress')
# the bbox widget
w_bbox = BBoxWidget(
    image = os.path.join(path, files[0]),
    classes=['apple', 'orange', 'pear']
)

# combine widgets into a container
w_container = widgets.VBox([
    w_progress,
    w_bbox,
])

Define the functions to process clicks on our Submit and Skip buttons.

You can use the widget's `on_skip` and `on_submit` methods as decorators on the functions.

In [ ]:
# when Skip button is pressed we move on to the next file
@w_bbox.on_skip
def skip():
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = os.path.join(path, image_file)
    # here we assign an empty list to bboxes but 
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    w_bbox.bboxes = [] 

# when Submit button is pressed we save current annotations
# and then move on to the next file
@w_bbox.on_submit
def submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    with open(annotations_path, 'w') as f:
        json.dump(annotations, f, indent=4)
    # move on to the next file
    skip()

Or if you don't like the decorators syntax you can assign the functions manually like this:

In [ ]:
# w_bbox.on_skip(skip)
# w_bbox.on_submit(submit);

Now we display the container widget and we are ready to annotate.

In [ ]:
w_container

The last image has more fruit kinds than we anticipated, so we can just add more classes to the list and keep going:

In [ ]:
w_bbox.classes = w_bbox.classes + ['lemon', 'grapefruit']

To verify the saved annotations we can look at the annotations file contents:

In [ ]:
with open(annotations_path, 'r') as f:
    print(f.read())

## Display special images

The `image` property of the widget expects either a local path to an image file or a web url.

If your image has a non-common format or requires special handling then the way to display it is to save it into a in-memory bytes buffer and assign the result to `widget.image_bytes`:

In [ ]:
# suppose this image is some fancy microscopy data
from PIL import Image
from io import BytesIO
image = Image.open('fruit/fruit2.jpg')

bytes_io = BytesIO()
image.save(bytes_io, format='jpeg')
widget.image_bytes = bytes_io.getvalue()

I'm open to suggestions on how to make this more user-friendly.

## Recording additional data

Sometimes you need more information about the object than just a location and a class label. For example, you might want to specify whether the object is in focus or blurred, record its size or other properties.

`BBoxWidget` lets you select a bbox (by clicking on it or with a `Tab`/`Shift-Tab` keyboard shortcut). The selected bbox is displayed with a thicker border. And its index is exposed in the `selected_index` widget trait. This makes it possible to use other widgets to edit additional properties of the selected bbox.

To facilitate this process `BBoxWidget` has an `attach` method that lets you attach another widget for editing an additional bbox property. 

For example, we want to apply a rating on a scale from 1 to 5 to every object in the image. Then we create a slider widget to edit the rating values:

```
w_rating = widgets.IntSlider(value=3, min=1, max=5, description='Rating')
```

And we attach it to the bbox widget.

```
w_bbox.attach(w_rating, name='rating')
```

As a result:

- Attached widget's value (`3` in this example) is used as the default `rating` value for new bboxes. Each newly created bbox will get a `rating` property with a value of `3` in addition to the usual `x`, `y`, `width`, `height` and `label`.
- When a bbox is selected the slider value will be set to bbox's `rating`.
- When you change the slider value then the new value is recorded in the selected bbox's `rating` property.
- When no bboxes are selected the attached widget will be disabled.

You can attach any number of widgets to the `BBoxWidget`. Displaying the attached widgets is left to you so you can make a layout that matches your use case.

### An example of attached widgets

Here's an example of using two attached widgets for two additional properties (`size` and `in_focus`). The output widget below will show live updates of bbox annotations as you play with the controls.

In [ ]:
w_bbox = BBoxWidget(
    image='fruit/fruit.jpg',
    classes=['apple', 'orange', 'pear'],
)
# a slider to record size
w_size = widgets.IntSlider(value=2, min=1, max=3, description='Size')
# a checkbox to record if the object is in focus
w_focus = widgets.Checkbox(value=True, description='Object is in focus')

w_bbox.attach(w_size, name='size')
w_bbox.attach(w_focus, name='in_focus')

# the output widget will show current state of bbox annotations
# as you play with the controls
w_out = widgets.Output()
def on_bbox_change(change):
    w_out.clear_output(wait=True)
    with w_out:
        print(json.dumps(change['new'], indent=4))
w_bbox.observe(on_bbox_change, names=['bboxes'])

w_container = widgets.VBox([
    w_bbox,
    widgets.HBox([
        w_size,
        w_focus,
    ]),
    w_out,
])
w_container